In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv("diabetes.csv")

In [3]:
df.to_csv("diabetes.csv",index=False)

In [4]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.corr()["Outcome"]

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [6]:
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
# on the basis of the target we are spliting our data
split=StratifiedShuffleSplit(n_splits=1,test_size=0.3,random_state=42)
for train_index,test_index in split.split(df,df["Outcome"]):
    strat_train_set=df.loc[train_index]
    strat_test_set=df.loc[test_index]

In [8]:
x_train=strat_train_set.drop("Outcome",axis=1)
y_train=strat_train_set["Outcome"]
x_test=strat_test_set.drop("Outcome",axis=1)
y_test=strat_test_set["Outcome"]

In [9]:
x_test.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
dtype: int64

In [10]:
x_train.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
dtype: int64

In [11]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
s_x_train=scaler.fit_transform(x_train)
s_x_test=scaler.transform(x_test)

In [12]:
import tensorflow
from tensorflow import keras 
from keras import Sequential
from keras.layers import Dense ,Dropout

In [13]:
model=Sequential()
model.add(Dense(32,kernel_initializer='he_uniform',activation='relu',input_dim=8))
model.add(Dense(units = 12, kernel_initializer = 'he_uniform',activation='relu'))
model.add(Dense(units = 1, kernel_initializer = 'glorot_uniform', activation = 'sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [14]:
model.fit(s_x_train,y_train,batch_size=12,epochs=50,validation_data=(s_x_test,y_test))

Epoch 1/50
45/45 [==============================] - 1s 10ms/step - loss: 0.8008 - accuracy: 0.4823 - val_loss: 0.7200 - val_accuracy: 0.5455
Epoch 2/50
45/45 [==============================] - 0s 6ms/step - loss: 0.6593 - accuracy: 0.6276 - val_loss: 0.6290 - val_accuracy: 0.6537
Epoch 3/50
45/45 [==============================] - 0s 7ms/step - loss: 0.5902 - accuracy: 0.6853 - val_loss: 0.5800 - val_accuracy: 0.6970
Epoch 4/50
45/45 [==============================] - 0s 6ms/step - loss: 0.5481 - accuracy: 0.7002 - val_loss: 0.5483 - val_accuracy: 0.7273
Epoch 5/50
45/45 [==============================] - 0s 6ms/step - loss: 0.5203 - accuracy: 0.7300 - val_loss: 0.5310 - val_accuracy: 0.7229
Epoch 6/50
45/45 [==============================] - 0s 6ms/step - loss: 0.5021 - accuracy: 0.7561 - val_loss: 0.5171 - val_accuracy: 0.7229
Epoch 7/50
45/45 [==============================] - 0s 6ms/step - loss: 0.4848 - accuracy: 0.7523 - val_loss: 0.5049 - val_accuracy: 0.7273
Epoch 8/50
45/45 [=

**Hyperparameter Tuning a Neural Network**

1. how to select appropriate optimizer
2. no. of nodes in a layer
3. how to select no. of layers
4. All in all one model 

**how to select appropriate optimizer**

In [15]:
import kerastuner as kt

In [16]:
def build_model(hp):
  model=Sequential()
  model.add(Dense(units=32,kernel_initializer = 'he_uniform',activation='relu',input_dim=8))
  model.add(Dense(units = 12, kernel_initializer = 'he_uniform',activation='relu'))
  model.add(Dense(1,kernel_initializer = 'glorot_uniform',activation='sigmoid'))
  optimizer=hp.Choice("optimizer",values=["adam","sgd","rmsprop","adadelta"])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [17]:
kt.GridSearch

keras_tuner.tuners.gridsearch.GridSearch

In [18]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='FOLDER1')

In [19]:
tuner.search(s_x_train,y_train,validation_data=(s_x_test,y_test),epochs=5)

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.6190476417541504

Best val_accuracy So Far: 0.761904776096344
Total elapsed time: 00h 00m 10s
INFO:tensorflow:Oracle triggered exit


In [20]:
tuner.get_best_hyperparameters()[0].values  # to get the best object

{'optimizer': 'adam'}

In [21]:
model=tuner.get_best_models(num_models=1)[0]

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 12)                396       
                                                                 
 dense_2 (Dense)             (None, 1)                 13        
                                                                 
Total params: 697
Trainable params: 697
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(s_x_train,y_train,batch_size=12,epochs=50,validation_data=(s_x_test,y_test),initial_epoch=6)

Epoch 7/50
45/45 [==============================] - 1s 8ms/step - loss: 0.4964 - accuracy: 0.7300 - val_loss: 0.4798 - val_accuracy: 0.7706
Epoch 8/50
45/45 [==============================] - 0s 6ms/step - loss: 0.4797 - accuracy: 0.7616 - val_loss: 0.4764 - val_accuracy: 0.7576
Epoch 9/50
45/45 [==============================] - 0s 6ms/step - loss: 0.4660 - accuracy: 0.7765 - val_loss: 0.4714 - val_accuracy: 0.7619
Epoch 10/50
45/45 [==============================] - 0s 5ms/step - loss: 0.4562 - accuracy: 0.7784 - val_loss: 0.4708 - val_accuracy: 0.7619
Epoch 11/50
45/45 [==============================] - 0s 5ms/step - loss: 0.4532 - accuracy: 0.7896 - val_loss: 0.4664 - val_accuracy: 0.7619
Epoch 12/50
45/45 [==============================] - 0s 5ms/step - loss: 0.4451 - accuracy: 0.7933 - val_loss: 0.4717 - val_accuracy: 0.7576
Epoch 13/50
45/45 [==============================] - 0s 5ms/step - loss: 0.4396 - accuracy: 0.7896 - val_loss: 0.4697 - val_accuracy: 0.7576
Epoch 14/50
45/4

**no. of nodes in a layer**

In [25]:
def build_model(hp):
  model=Sequential()
  units=hp.Int('units',min_value=8,max_value=128,step=8)        # hp.Int(initia_value,last_value,step)
  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  

  return model

In [26]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='FOLDER2')

In [27]:
tuner.search(s_x_train,y_train,validation_data=(s_x_test,y_test),epochs=5)

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7402597665786743

Best val_accuracy So Far: 0.7402597665786743
Total elapsed time: 00h 00m 12s
INFO:tensorflow:Oracle triggered exit


In [28]:
tuner.get_best_hyperparameters()[0].values  # to get the best object

{'units': 112}

In [29]:
model=tuner.get_best_models(num_models=1)[0]

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 112)               1008      
                                                                 
 dense_1 (Dense)             (None, 1)                 113       
                                                                 
Total params: 1,121
Trainable params: 1,121
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.fit(s_x_train,y_train,batch_size=12,epochs=50,validation_data=(s_x_test,y_test),initial_epoch=5)

Epoch 6/50
45/45 [==============================] - 1s 8ms/step - loss: 0.4898 - accuracy: 0.7821 - val_loss: 0.4755 - val_accuracy: 0.7446
Epoch 7/50
45/45 [==============================] - 0s 6ms/step - loss: 0.4713 - accuracy: 0.7858 - val_loss: 0.4674 - val_accuracy: 0.7489
Epoch 8/50
45/45 [==============================] - 0s 6ms/step - loss: 0.4605 - accuracy: 0.7970 - val_loss: 0.4640 - val_accuracy: 0.7532
Epoch 9/50
45/45 [==============================] - 0s 5ms/step - loss: 0.4551 - accuracy: 0.7877 - val_loss: 0.4644 - val_accuracy: 0.7446
Epoch 10/50
45/45 [==============================] - 0s 5ms/step - loss: 0.4509 - accuracy: 0.7933 - val_loss: 0.4635 - val_accuracy: 0.7359
Epoch 11/50
45/45 [==============================] - 0s 6ms/step - loss: 0.4458 - accuracy: 0.7970 - val_loss: 0.4650 - val_accuracy: 0.7403
Epoch 12/50
45/45 [==============================] - 0s 5ms/step - loss: 0.4443 - accuracy: 0.7952 - val_loss: 0.4650 - val_accuracy: 0.7359
Epoch 13/50
45/45

**how to select no. of layers**

In [53]:
def build_model(hp):
    
    model = Sequential()
    
    model.add(Dense(units=72,activation='relu',input_dim=8)) # defining my input layer
    
    for i in range (hp.Int('num_layers',min_value=1,max_value=10)):
        model.add(Dense(units=72,activation='relu'))
        
    model.add(Dense(1, activation='sigmoid'))
    model.compile
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

    return model

In [54]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='FOLDER3')

In [55]:
tuner.search(s_x_train,y_train,validation_data=(s_x_test,y_test),epochs=5)

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 16s
INFO:tensorflow:Oracle triggered exit


In [56]:
tuner.get_best_hyperparameters()[0].values  # to get the best object

{'num_layers': 6}

In [57]:
model=tuner.get_best_models(num_models=1)[0]

In [58]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 72)                648       
                                                                 
 dense_1 (Dense)             (None, 72)                5256      
                                                                 
 dense_2 (Dense)             (None, 72)                5256      
                                                                 
 dense_3 (Dense)             (None, 72)                5256      
                                                                 
 dense_4 (Dense)             (None, 72)                5256      
                                                                 
 dense_5 (Dense)             (None, 72)                5256      
                                                                 
 dense_6 (Dense)             (None, 72)                5

In [59]:
model.fit(s_x_train,y_train,batch_size=12,epochs=50,validation_data=(s_x_test,y_test),initial_epoch=6)

Epoch 7/50
45/45 [==============================] - 1s 11ms/step - loss: 0.4775 - accuracy: 0.7654 - val_loss: 0.5998 - val_accuracy: 0.7273
Epoch 8/50
45/45 [==============================] - 0s 8ms/step - loss: 0.4570 - accuracy: 0.7914 - val_loss: 0.5873 - val_accuracy: 0.7359
Epoch 9/50
45/45 [==============================] - 0s 9ms/step - loss: 0.4550 - accuracy: 0.7933 - val_loss: 0.5032 - val_accuracy: 0.7489
Epoch 10/50
45/45 [==============================] - 0s 8ms/step - loss: 0.4316 - accuracy: 0.7896 - val_loss: 0.5619 - val_accuracy: 0.7403
Epoch 11/50
45/45 [==============================] - 0s 9ms/step - loss: 0.4246 - accuracy: 0.8175 - val_loss: 0.5016 - val_accuracy: 0.7576
Epoch 12/50
45/45 [==============================] - 0s 9ms/step - loss: 0.4079 - accuracy: 0.8119 - val_loss: 0.5082 - val_accuracy: 0.7446
Epoch 13/50
45/45 [==============================] - 0s 8ms/step - loss: 0.3905 - accuracy: 0.8268 - val_loss: 0.5273 - val_accuracy: 0.7619
Epoch 14/50
45/

In [60]:
# model overfittling

# All in all one model

In [46]:
def build_model(hp):
    
    model = Sequential()
    counter=0 
    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        if counter==0:
            model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                           activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid'])
                            ,input_dim=8
                           )
                     )
            model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
            
        else:
                  
            model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                           activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid'])
                          
                           )
                     )
            model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        counter+=1
    
        
        
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=hp.Choice("optimizer",values=["adam","sgd","rmsprop","adadelta"]),loss='binary_crossentropy',metrics=['accuracy'])

    return model

In [47]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='FOLDER4')

In [48]:
tuner.search(s_x_train,y_train,validation_data=(s_x_test,y_test),epochs=5)

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7359307408332825

Best val_accuracy So Far: 0.7359307408332825
Total elapsed time: 00h 00m 13s
INFO:tensorflow:Oracle triggered exit


In [49]:
tuner.get_best_hyperparameters()[0].values  # to get the best object

{'num_layers': 1,
 'units0': 24,
 'activation0': 'tanh',
 'dropout0': 0.2,
 'optimizer': 'rmsprop',
 'units1': 16,
 'activation1': 'relu',
 'dropout1': 0.5,
 'units2': 8,
 'activation2': 'sigmoid',
 'dropout2': 0.9,
 'units3': 48,
 'activation3': 'relu',
 'dropout3': 0.5,
 'units4': 32,
 'activation4': 'sigmoid',
 'dropout4': 0.3,
 'units5': 56,
 'activation5': 'tanh',
 'dropout5': 0.1,
 'units6': 96,
 'activation6': 'tanh',
 'dropout6': 0.8,
 'units7': 8,
 'activation7': 'sigmoid',
 'dropout7': 0.5}

In [50]:
model=tuner.get_best_models(num_models=1)[0]


In [51]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                216       
                                                                 
 dropout (Dropout)           (None, 24)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                                 
Total params: 241
Trainable params: 241
Non-trainable params: 0
_________________________________________________________________


In [52]:
model.fit(s_x_train,y_train,batch_size=12,epochs=100,validation_data=(s_x_test,y_test),initial_epoch=6)

Epoch 7/100
45/45 [==============================] - 1s 10ms/step - loss: 0.5172 - accuracy: 0.7542 - val_loss: 0.5116 - val_accuracy: 0.7489
Epoch 8/100
45/45 [==============================] - 0s 5ms/step - loss: 0.5069 - accuracy: 0.7337 - val_loss: 0.4992 - val_accuracy: 0.7446
Epoch 9/100
45/45 [==============================] - 0s 5ms/step - loss: 0.4919 - accuracy: 0.7747 - val_loss: 0.4928 - val_accuracy: 0.7403
Epoch 10/100
45/45 [==============================] - 0s 6ms/step - loss: 0.4881 - accuracy: 0.7672 - val_loss: 0.4892 - val_accuracy: 0.7359
Epoch 11/100
45/45 [==============================] - 0s 8ms/step - loss: 0.4703 - accuracy: 0.7803 - val_loss: 0.4871 - val_accuracy: 0.7316
Epoch 12/100
45/45 [==============================] - 0s 6ms/step - loss: 0.4856 - accuracy: 0.7635 - val_loss: 0.4856 - val_accuracy: 0.7359
Epoch 13/100
45/45 [==============================] - 0s 5ms/step - loss: 0.4752 - accuracy: 0.7803 - val_loss: 0.4855 - val_accuracy: 0.7359
Epoch 14

Epoch 65/100
45/45 [==============================] - 0s 7ms/step - loss: 0.4417 - accuracy: 0.8063 - val_loss: 0.4807 - val_accuracy: 0.7403
Epoch 66/100
45/45 [==============================] - 0s 6ms/step - loss: 0.4614 - accuracy: 0.7840 - val_loss: 0.4804 - val_accuracy: 0.7403
Epoch 67/100
45/45 [==============================] - 0s 6ms/step - loss: 0.4540 - accuracy: 0.7803 - val_loss: 0.4801 - val_accuracy: 0.7446
Epoch 68/100
45/45 [==============================] - 0s 6ms/step - loss: 0.4555 - accuracy: 0.7896 - val_loss: 0.4795 - val_accuracy: 0.7403
Epoch 69/100
45/45 [==============================] - 0s 6ms/step - loss: 0.4455 - accuracy: 0.7914 - val_loss: 0.4804 - val_accuracy: 0.7359
Epoch 70/100
45/45 [==============================] - 0s 6ms/step - loss: 0.4549 - accuracy: 0.7970 - val_loss: 0.4801 - val_accuracy: 0.7359
Epoch 71/100
45/45 [==============================] - 0s 6ms/step - loss: 0.4374 - accuracy: 0.7970 - val_loss: 0.4804 - val_accuracy: 0.7359
Epoch 